In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
semana = input("Ingrese Semana:")

Ingrese Semana:42


In [3]:
mainpath = "C:/Users/mromo/Desktop/Rappi/Datasets/S"+semana+"/Data"
ganancias_path = "Detalle Ganancia S"+semana+".csv"
deuda_path = "Detalle Deuda S"+semana+".csv"
garantias_path = "Garantias Turnos.xlsx"
conexion_path = "Info Conexion.csv"
ordenes_path = "Detalle Ordenes S"+semana+".csv"

fp_earn = os.path.join(mainpath,ganancias_path)
fp_debt = os.path.join(mainpath,deuda_path)
fp_garantias = os.path.join(mainpath,garantias_path)
fp_conexion = os.path.join(mainpath,conexion_path)
fp_ordenes = os.path.join(mainpath,ordenes_path)

In [4]:
df_ganancia = pd.read_csv(fp_earn, parse_dates = ['created_at'])
df_conexion = pd.read_csv(fp_conexion, parse_dates = ['fecha'])
df_ldam = pd.read_excel(fp_garantias, sheet_name = "La Dehesa AM")
df_ldpm = pd.read_excel(fp_garantias, sheet_name = "La Dehesa PM")
df_autos = pd.read_excel(fp_garantias, sheet_name = "Autos")
df_vina = pd.read_excel(fp_garantias, sheet_name = "Viña")
df_ordenes = pd.read_csv(fp_ordenes, parse_dates = ['CREATED_AT'])

In [5]:
ldam = df_ldam['ID'].tolist()
ldpm = df_ldpm['ID'].tolist()
autos = df_autos['ID'].tolist()
vina = df_vina['ID'].tolist()

In [6]:
rts_gar = ldam + ldpm
rts_gar = rts_gar + autos
rts_gar = rts_gar + vina
ld = ldam + ldpm

In [7]:
df_conexion = df_conexion[df_conexion['skid'].isin(rts_gar)]

In [8]:
def calculoGarantia(tiempo, valor):
    if(tiempo >= 30):
        return valor

In [9]:
def garantiasHorarias(df_conexion, horaA, horaC, dias, lista, valor):
    dft1 = df_conexion[df_conexion['hora']>=horaA]
    dft1 = dft1[dft1['hora']<horaC]
    dft1.insert(2,'Dia',dft1['fecha'].apply(lambda x: x.strftime("%w")))
    dft1 = dft1[dft1['skid'].isin(lista)]
    dft1 = dft1[dft1['Dia'].astype(int).isin(dias)]
    dft1.insert(8, "Garantia", dft1['time'].apply(lambda x: calculoGarantia(x,valor)))
    dft1 = dft1.fillna(0)
    dft1 = dft1.groupby(['skid','fecha','Dia']).sum().reset_index()
    dft1.insert(8,'Garantia Efectiva',np.maximum(0,dft1.Garantia-dft1.ganancia))
    return dft1

In [10]:
df_ordenes.head()

,STOREKEEPER_ID,CREATED_AT,NAME,MODEL_ID,ADDRESS,VERTICAL,TOTAL,DESCRIPTION
0,10042,2018-10-19 23:03:52.871,service_cost,332858,Marín 145,restaurantes,2400.0,NaN
1,10042,2018-10-19 23:03:52.871,service_cost,332858,Marín 145,restaurantes,2400.0,NaN
2,10042,2018-10-19 23:03:52.871,service_cost,332858,Marín 145,restaurantes,2400.0,NaN
3,10042,2018-10-19 23:03:52.871,service_cost,332858,Marín 145,restaurantes,2400.0,NaN
4,10042,2018-10-19 23:03:52.891,surcharge_paid_for_rappi,332858,Marín 145,restaurantes,525.0,NaN


In [11]:
df_ganancia.insert(2,'fecha',df_ganancia['created_at'].apply(lambda x: x.strftime("%Y-%M-%d")))
df_ganancia.insert(3,'hora',df_ganancia['created_at'].apply(lambda x: x.strftime("%H")))
df_ganancia = df_ganancia.rename(columns ={'storekeeper_id':'skid'})

In [12]:
df_ganancia.head()

,skid,created_at,fecha,hora,name,model_id,total,description
0,10042,2018-10-19 23:03:52.871,2018-03-19,23,service_cost,332858,2400.0,NaN
1,10042,2018-10-19 23:03:52.891,2018-03-19,23,surcharge_paid_for_rappi,332858,525.0,NaN
2,10042,2018-10-19 23:42:51.978,2018-42-19,23,service_cost,333059,2400.0,NaN
3,10042,2018-10-19 23:42:51.992,2018-42-19,23,tip,333059,500.0,NaN
4,10042,2018-10-19 23:42:52.006,2018-42-19,23,surcharge_paid_for_rappi,333059,575.0,NaN


In [13]:
df_ganancia['hora'] = df_ganancia['hora'].apply(lambda x: int(x))

In [14]:
#Garantia la Dehesa 9 a 9, Lunes a Domingo
dft1 = df_conexion[df_conexion['hora']>8] 
dft1 = dft1[dft1['hora']<22]
dft1.insert(2,'Dia',dft1['fecha'].apply(lambda x: x.strftime("%w")))
dft_autos1 = dft1[dft1['Dia'].astype(int)==6]
dft_autos1 = dft_autos1[dft_autos1['skid'].isin(autos)]
dft_vina = dft1[dft1['skid'].isin(vina)]
dft1 = dft1[dft1['skid'].isin(ldam)]
dft1.insert(8, "Garantia", dft1['time'].apply(lambda x: calculoGarantia(x,2800)))
dft1 = dft1.fillna(0)

In [15]:
dft1 = dft1.groupby(['skid','fecha','Dia']).sum().reset_index()
dft1.insert(8,'Garantia Efectiva',np.maximum(0,dft1.Garantia-dft1.ganancia))

In [16]:
#Garantia la Dehesa 9 a 9, Lunes a Domingo
dias = [0,1,2,3,4,5,6]
dft_ldam = garantiasHorarias(df_conexion,9,21, dias, ldam, 2800)

In [17]:
dft_vina.insert(8, "Garantia", dft_vina['time'].apply(lambda x: calculoGarantia(x,2800)))
dft_vina = dft_vina.groupby(['skid','fecha','Dia']).sum().reset_index()
dft_vina.insert(8,'Garantia Efectiva', np.minimum(25200,dft_vina.Garantia-dft_vina.ganancia))

In [18]:
dft_vResumen = dft_vina.groupby('skid').sum()

In [19]:
#Garantia la dehesa 12 a 12, Lunes a Jueves y Domingo
dft2 = df_conexion[df_conexion['hora']>11]
dft2 = dft2[dft2['hora']<24]
dft2.insert(2,'Dia',dft2['fecha'].apply(lambda x: x.strftime("%w")))
dft2.drop((dft2['Dia'].astype(int)==5).index)
dft2.drop((dft2['Dia'].astype(int)==6).index)
dft2 = dft2[dft2['skid'].isin(ldpm)]
dft2.insert(8, "Garantia", dft2['time'].apply(lambda x: calculoGarantia(x,2800)))
dft2 = dft2.fillna(0)
dft2 = dft2.groupby(['skid','fecha','Dia']).sum().reset_index()
dft2.insert(8,'Garantia Efectiva',np.maximum(0,dft2.Garantia-dft2.ganancia))

In [20]:
#Garantia la dehesa 15 a 3, Viernes y Sabado
dft4 = df_conexion[df_conexion['hora']>14] 
dft5 = dft4[dft4['hora']<3] 
dft6 = pd.merge(dft4,dft5,how = 'outer', on=['skid', 'fecha','hora','time', 'orders', 'ganancia', 'tenia_cupo'])
dft6 = dft6.fillna(0)
dft6.insert(2,'Dia',dft6['fecha'].apply(lambda x: x.strftime("%w")))
dfdummy1 = dft6[dft6['Dia'].astype(int)==4]
dfdummy2 = dft6[dft6['Dia'].astype(int)==5]
dft10 = pd.merge(dfdummy1, dfdummy2, how = 'outer', on=['skid', 'fecha','hora','time', 'orders', 'ganancia', 'tenia_cupo','Dia'])
dft10 = dft10[dft10['skid'].isin(ldpm)]
dft10.insert(8, "Garantia", dft10['time'].apply(lambda x: calculoGarantia(x,2800)))

In [21]:
dft10 = dft10.groupby(['skid','fecha','Dia']).sum().reset_index()
dft10.insert(8,'Garantia Efectiva', np.maximum(0,dft10.Garantia-dft10.ganancia))

In [22]:
#Garantia la Dehesa 12 a 12, Lunes a Jueves y Domingo y 15 a 3, Viernes y Sabado
dias = [0,1,2,3,4]
df_ldpm1 = garantiasHorarias(df_conexion, 12, 24, dias, ldpm, 2800)
df_ldpm2 = garantiasHorarias(df_conexion, 12, 24, [5,6], ldpm, 2800)
dft_ldpm = pd.merge(df_ldpm1, df_ldpm2, how='outer', on=df_ldpm1.columns.tolist())

In [23]:
#Garantia 10 a 10, Autos Domingo
dft3 = df_conexion[df_conexion['hora']>9] 
dft3 = dft3[dft3['hora']<22] 
dft3 = dft3.fillna(0)
dft3.insert(2,'Dia',dft3['fecha'].apply(lambda x: x.strftime("%w")))
dft_autos2 = dft3[dft3['Dia'].astype(int)==0]
dft_autos2 = dft_autos2[dft_autos2['skid'].isin(autos)]
dft_autos = pd.merge(dft_autos1, dft_autos2, how = 'outer', on = dft_autos1.columns.tolist())

In [24]:
dft_autos = dft_autos.groupby(['skid','fecha','Dia']).sum().reset_index()
dft_autos.insert(7,'Garantia Efectiva', 40000-dft_autos.ganancia)

In [25]:
df_ordLD = df_ordenes[df_ordenes['STOREKEEPER_ID'].isin(ld)]
df_ordLD = df_ordLD[df_ordLD['NAME']=='service_cost']
df_ordLD = df_ordLD.groupby(['STOREKEEPER_ID','MODEL_ID', 'NAME', 'ADDRESS','CREATED_AT']).sum()

In [26]:
#Garantia 10 a 10 Domingo y 9 a 9 Sabado Autos
df_autos1 = garantiasHorarias(df_conexion, 9, 21, [6], autos, 3334)
df_autos2 = garantiasHorarias(df_conexion, 10, 22, [0], autos, 3334)
df_autos = pd.merge(df_autos1, df_autos2, how = 'outer', on = df_autos1.columns.tolist())
df_autos2

,skid,fecha,Dia,hora,time,orders,ganancia,Garantia,Garantia Efectiva
0,11956,2018-10-21,0,186,691.466667,12.0,25200.0,40008,14808.0


In [27]:
#Garantias Viña 3 Turnos
dias = [0,1,2,3,4,5,6]
dft_vina1 = garantiasHorarias(df_conexion,11,23, dias, vina, 2800)

In [28]:
writer = pd.ExcelWriter('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Garantias S'+semana+'.xlsx')
dft1.to_excel(writer,'Garantia Dehesa 9-9')
dft2.to_excel(writer,'Garantia Dehesa PM 12-24')
dft10.to_excel(writer,'Garantia Dehesa PM 15 a 3')
dft_autos.to_excel(writer,'Garantias Autos')
dft_vina.to_excel(writer,'Garantias Viña')
writer.save()
writer.close()
print("Garantias Calculadas en C:/Users/mromo/Desktop/Rappi/Datasets/S"+semana+"/Garantias S"+semana+".xlsx")

Garantias Calculadas en C:/Users/mromo/Desktop/Rappi/Datasets/S42/Garantias S42.xlsx


In [29]:
writer = pd.ExcelWriter('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Garantias S'+semana+' v2.xlsx')
dft_ldam.to_excel(writer,'Garantia Dehesa AM')
dft_ldpm.to_excel(writer,'Garantia Dehesa PM')
df_autos.to_excel(writer,'Garantia Autos')
dft_vina.to_excel(writer,'Garantias Viña')
writer.save()
writer.close()
print("Garantias Calculadas en C:/Users/mromo/Desktop/Rappi/Datasets/S"+semana+"/Garantias S"+semana+" v2.xlsx")

Garantias Calculadas en C:/Users/mromo/Desktop/Rappi/Datasets/S42/Garantias S42 v2.xlsx


In [30]:
df_ordLD.to_excel('C:/Users/mromo/Desktop/Rappi/Datasets/S'+semana+'/Ordenes LD S'+semana+'.xlsx')